## This is a standalone notebook, used for development purposes

In [ ]:
## setup variables

In [ ]:
## setup_secrets

# use `load_env` and define a sample .env template to demonstrate.

In [ ]:
# get_compute

In [ ]:
# get_environment

In [ ]:
# create a command job

In [ ]:
# wait for the command job to finish

In [ ]:
# orchestrate !!

# get_compute
# get_environment
# command job
# wait for the command job to get over. (optional)